In [18]:
import pandas as pd
import ast

In [21]:
df=pd.read_csv(r'C:\Users\Yasaman\Arab_spring_scholarly_attention\Validating\final_annotated.csv')

In [12]:
df.shape


(1000, 18)

In [13]:
df['SampleGroup'].value_counts()

SampleGroup
field_20             520
with_mention         280
with_mention_arab    200
Name: count, dtype: int64

In [17]:
df[df['SampleGroup']=='field_20']['Field'].value_counts()

Field
Immunology and Microbiology                     20
Biochemistry, Genetics and Molecular Biology    20
Pharmacology, Toxicology and Pharmaceutics      20
Energy                                          20
Economics, Econometrics and Finance             20
Social Sciences                                 20
Engineering                                     20
Neuroscience                                    20
Physics and Astronomy                           20
Materials Science                               20
Nursing                                         20
Business, Management and Accounting             20
Agricultural and Biological Sciences            20
Chemistry                                       20
Dentistry                                       20
Mathematics                                     20
Chemical Engineering                            20
Psychology                                      20
Health Professions                              20
Earth and Planetary Scien

In [22]:
df['union_annotation'] = df['union_annotation'].apply(ast.literal_eval)
df['intersection_annotation'] = df['intersection_annotation'].apply(ast.literal_eval)
#df['Mentions'] = df['Mentions'].apply(ast.literal_eval)


In [ ]:
df['annotator_agreement'].value_counts()

annotator_agreement
True     891
False    109
Name: count, dtype: int64